In [0]:
# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib

In [0]:
dbutils.fs.ls(“/FileStore/tables”)

File "<command-2152267213897291>" , line 1 
 dbutils.fs.ls(“/FileStore/tables”) 
 ^
 SyntaxError : invalid character in identifier

In [0]:
# Specify file type to be csv
file_type = "csv"
# Indicates file has first row as the header
first_row_is_header = "true"
# Indicates file has comma as the delimeter
delimiter = ","
# Read the CSV file to spark dataframe
aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load("/FileStore/tables/authentication_credentials.csv")

In [0]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [0]:
# AWS S3 bucket name
AWS_S3_BUCKET = "user-0e0816526d11-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/user-0e0816526d11-bucket"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

Out[39]: True

In [0]:
# list the topics stored on the mounted S3 bucket
display(dbutils.fs.ls("/mnt/user-0e0816526d11-bucket/topics"))

path,name,size,modificationTime
dbfs:/mnt/user-0e0816526d11-bucket/topics/0e0816526d11.geo/,0e0816526d11.geo/,0,1701815495494
dbfs:/mnt/user-0e0816526d11-bucket/topics/0e0816526d11.pin/,0e0816526d11.pin/,0,1701815495494
dbfs:/mnt/user-0e0816526d11-bucket/topics/0e0816526d11.user/,0e0816526d11.user/,0,1701815495494


In [0]:
# list of topic suffixes
topic = [".geo", ".pin", ".user"]

def read_data_from_mounted_s3_bucket(topic):
    # create path to topic files
    file_location = f"/mnt/user-0e0816526d11-bucket/topics/0e0816526d11{topic}/partition=0/*.json"
    # specify file type
    file_type = "json"
    # Ask Spark to infer the schema
    infer_schema = "true"
    # load JSONs from mounted S3 bucket to Spark dataframe
    df = spark.read.format(file_type) \
        .option("inferSchema", infer_schema) \
        .load(file_location)
    return df

for item in topic:
    # create statement strings for naming and displaying dataframes
    make_df = f"df_{item[1:]} = read_data_from_mounted_s3_bucket('{item}')"
    display_df = f"display(df_{item[1:]})"
    # execute statements
    exec(make_df)
    exec(display_df)

country,ind,index,latitude,longitude,timestamp
Cocos (Keeling) Islands,10794,null,-89.5236,-154.567,2022-01-01 02:26:50
"[""country""]",null,List(index),"[""latitude""]","[""longitude""]","[""timestamp""]"
"[""country""]",null,List(index),"[""latitude""]","[""longitude""]","[""timestamp""]"
"[""country""]",null,List(index),"[""latitude""]","[""longitude""]","[""timestamp""]"
"[""country""]",null,List(index),"[""latitude""]","[""longitude""]","[""timestamp""]"
Cote d'Ivoire,2923,null,-84.6302,-164.507,2019-09-08 22:53:09
French Guiana,8304,null,-28.8852,-164.87,2019-09-13 04:50:29
Cote d'Ivoire,4315,null,-45.8508,66.1003,2019-12-15 03:51:28
French Guiana,8304,null,-28.8852,-164.87,2019-09-13 04:50:29
French Guiana,8304,null,-28.8852,-164.87,2019-09-13 04:50:29


category,description,downloaded,follower_count,image_src,index,is_image_or_video,poster_name,save_location,tag_list,title,unique_id
finance,"If you love budgeting, make sure to give Dave Ramsey's 7 Baby Steps a try. Follow these steps to begin your debt snowball, build an emergency fund, invest and reach riches. I ca…",1,26k,https://i.pinimg.com/originals/1e/9d/90/1e9d906e4e150e3b95187f3b76ea7c71.png,5494,image,"Living Low Key | Save Money, Make Money, & Frugal Living",Local save in /data/finance,"Financial Peace,Financial Tips,Saving Money Quotes,Total Money Makeover,Budgeting Finances,Money Management,Wealth Management,Personal Finance,Making Ideas",Dave Ramsey's 7 Baby Steps: What Are They And Will They Work For You,8fb2af68-543b-4639-8119-de33d28706ed
home-decor,"Holiday mantle decor, Christmas decor, metallic mercury glass style Christmas trees, eucalyptus vine, evergreen pine branches, white neutral holiday decor, cozy mantle for the h…",1,83k,https://i.pinimg.com/originals/9d/82/1a/9d821a80acd8f90c16454e978bd9b115.jpg,6145,image,Stylin by Aylin,Local save in /data/home-decor,"Winter Home Decor,Christmas Living Room Decor,Living Room Decor Cozy,Christmas Decor,Cozy Fireplace,Rustic Fireplace Decor,Fireplace Decorations,Rustic Room,House Decorations",HOLIDAY MANTLE DECOR - @AMAZON & @TARGET FINDS,82e13a07-db99-43a3-b1c0-89a4b75821da
education,"Podcasts for Teachers or Parents of Teenagers: Teaching teens middle school and high school can feel joyful and rewarding most days, but can also frustrate you with one challeng…",1,25k,https://i.pinimg.com/originals/50/19/31/501931a27ee4d076658980851b995b2c.jpg,4315,image,Math Giraffe,Local save in /data/education,"Middle School Classroom,High School Students,High School Teachers,Middle School Tips,High School Counseling,Ela Classroom,High School Science,Future Classroom,Google Classroom",Podcasts for Teachers or Parents of Teenagers,21b59ba9-829d-4c33-8c27-4cd4c56d26b8
education,"Podcasts for Teachers or Parents of Teenagers: Teaching teens middle school and high school can feel joyful and rewarding most days, but can also frustrate you with one challeng…",1,25k,https://i.pinimg.com/originals/50/19/31/501931a27ee4d076658980851b995b2c.jpg,4315,image,Math Giraffe,Local save in /data/education,"Middle School Classroom,High School Students,High School Teachers,Middle School Tips,High School Counseling,Ela Classroom,High School Science,Future Classroom,Google Classroom",Podcasts for Teachers or Parents of Teenagers,21b59ba9-829d-4c33-8c27-4cd4c56d26b8
event-planning,Sacramento California Wedding 2 Chic Events & Design Jodi Yorston Photography Wilson Vineyards Barn Miosa Couture Yellow Barn Vineyard Outdoor Candles DIY,1,6M,https://i.pinimg.com/originals/7e/45/90/7e45905fefa36347e83333fd6d091140.jpg,5069,image,Style Me Pretty,Local save in /data/event-planning,"60th Anniversary Parties,Anniversary Decorations,Golden Anniversary,25th Wedding Anniversary,Anniversary Pictures,Anniversary Ideas,Birthday Decorations,Event Planning Design,Event Design",The Vault: Curated & Refined Wedding Inspiration,b75b6f87-deb3-444f-b29e-ce9161b2df49
home-decor,"Beautiful and easy ways to update every room in your home with fall decor. Plus, my favorite finds in fall decor for 2020!",1,92k,https://i.pinimg.com/originals/e5/ae/dc/e5aedc14ce557e3a69f672e0f8c88f6e.png,6063,image,"Kristen | Lifestyle, Mom Tips & Teacher Stuff Blog",Local save in /data/home-decor,"Fall Home Decor,Autumn Home,Fall Decor Outdoor,Front Porch Fall Decor,Home Decor Ideas,Porch Ideas For Fall,Fall Outdoor Decorating,Decorating Ideas For Fall,Fall Front Doors",41 Gorgeous Fall Decor Ideas For Your Home - Chaylor & Mads,60693727-4927-4bd6-a8c5-096a392d63e6
diy-and-crafts,HELPFUL RESOURCES – Check out my resin colorants resources page here with links to all the products mentioned in this article (and more). Let me know if you have any that you lo…,1,6k,https://i.pinimg.com/originals/d4/12/78/d4127833023ca32600571ddca16f1556.jpg,3454,ima

age,date_joined,first_name,ind,index,last_name
"[""age""]","[""date_joined""]","[""first_name""]",null,List(index),"[""last_name""]"
"[""age""]","[""date_joined""]","[""first_name""]",null,List(index),"[""last_name""]"
"[""age""]","[""date_joined""]","[""first_name""]",null,List(index),"[""last_name""]"
"[""age""]","[""date_joined""]","[""first_name""]",null,List(index),"[""last_name""]"
21,2015-11-10 09:27:42,Andrea,8731,null,Alexander
21,2015-11-10 09:27:42,Andrea,8731,null,Alexander
36,2015-12-20 16:38:13,Michelle,4315,null,Prince
32,2017-10-10 20:09:33,Christian,10625,null,Lang
32,2016-04-02 03:51:23,Brittany,1313,null,Jones
34,2016-12-22 00:02:02,Thomas,10794,null,Turner


In [0]:
dbutils.fs.unmount("/mnt/user-0e0816526d11-bucket")

/mnt/user-0e0816526d11-bucket has been unmounted.
Out[34]: True